#Inspiration

https://dev.to/devdevcharlie/play-street-fighter-with-body-movements-using-arduino-and-tensorflow-js-4kbi

Built in JS with Johnny-Five and TensorFlow.js, goal is to port it to TensorFlow Lite Micro and run ML inferencing directly on an Arduino.

# Arduino Development Environment setup

1. Download the Arduino IDE
1. Open the Arduino IDE
1. Install Nano 33 BLE board support via: `Tools -> Board: ... -> Board Manager ...`
1. Install the `Arduino_LSM9DS1` library via: `Sketch -> Include Library -> Manage Libraries ...`
1. Download: https://storage.googleapis.com/tensorflow-nightly/github/tensorflow/tensorflow/lite/experimental/micro/tools/make/gen/arduino_x86_64/prj/tensorflow_lite.zip
1. Import downloaded library: `Sketch -> Include Library -> Add .ZIP library ...`

# Hardware setup

1. Insert Arduino Nano 33 BLE Sense board into mini breadboard
1. Insert button between the GND and D3 pin
1. Plug in micro USB cable to board and into PC

# Testing hardware

1. In the Arduino IDE, select: `Tools -> Board: ... -> Nano 33 BLE`
1. Select the Serial Port for the board: `Tools -> Port -> `
1. Open `Button` example: `File -> Examples -> 02.Digital -> Button`
1. Change `const int buttonPin = 2;` line to `const int buttonPin = 3;` (to match the pin we placed the button on)
1. Change the `pinMode(buttonPin, INPUT);` line to `pinMode(buttonPin, INPUT_PULLUP);` (as we will use an internal pullup resistor)
1. Press the `Upload` button
1. After uploading, the orange LED should remain on.
1. Press the button, the LED should go off

# Visualize IMU Data

1. Download and upload the https://github.com/sandeepmistry/aimldevfest-workshop-2019/blob/master/ArduinoSketches/IMU_Capture/IMU_Capture.ino sketch to the board

1. Open the Serial Monitor: `Tools -> Serial Monitor`
1. Press the button, IMU data will be captured and outputted for 1 second
1. Close the Serial Monitor window
1. Open the Serial Plotter: `Tools -> Serial Plotter`
1. Press the button, and perform a gesture
1. You'll see a graph of the data capture
1. Repeat capturing various gestures to get a sense of what the training data looks like
1. Close the Serial Plotter

# Gather Data Training

1. Press the reset button on the board
1. Open the Serial Monitor: Tools -> Serial Monitor
1. Push the button, make a punch gesture
1. Repeat 10x
1. Copy and paste the data from the serial output to punch.csv
1. Close the Serial Monitor
1. Press the reset button on the board
1. Open the Serial Monitor: Tools -> Serial Monitor
1. Push the button, flex
1. Repeat 10x
1. Copy and paster the serial output to flex.csv



# Setup Python Environment 

The next cell sets up the dependencies in required for the notebook, run it.

In [0]:
# Setup environment
!apt-get -qq install xxd
!pip install tensorflow==2.0.0-rc1

# Upload Data

1. Open the panel on the left side of Colab by clicking on the __>__
1. Select the files tab
1. Drag `punch.csv` and `flex.csv` files from your computer to the tab to upload them into colab.


# TODO
Add a test for uploaded models here

List the file names in /content/*.csv

Throw an error if we don't see any

# Train Neural Network

The next cell parses the csv and trains a fully connected neural network.

Update the `GESTURES` list with the gesture data you've collected in `.csv` format.

The models performance vs validation will also be graphed.


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

print("TensorFlow version =", tf.__version__)

# set a fixed random seed for reproducibility
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

# the list of gestures that data is available for
GESTURES = [
    "punch",
    "flex",
]

SAMPLES_PER_GESTURE = 119

NUM_GESTURES = len(GESTURES)

# create a one-hot encoded matrix that is used in the output
ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)

inputs = []
outputs = []

# read each csv file and push an input and output
for gesture_index in range(NUM_GESTURES):
  gesture = GESTURES[gesture_index]
  print(gesture_index, gesture)
  
  output = ONE_HOT_ENCODED_GESTURES[gesture_index]
  
  df = pd.read_csv("/content/" + gesture + ".csv")
  
  # calculate the number of gesture recordings in the file
  num_recordings = int(df.shape[0] / SAMPLES_PER_GESTURE)
  
  print(num_recordings)
  
  for i in range(num_recordings):
    tensor = []
    for j in range(SAMPLES_PER_GESTURE):
      index = i * SAMPLES_PER_GESTURE + j
      tensor += [
          df['aX'][index],
          df['aY'][index],
          df['aZ'][index],
          df['gX'][index],
          df['gY'][index],
          df['gZ'][index]
      ]

    inputs.append(tensor)
    outputs.append(output)

    
inputs = np.array(inputs)
outputs = np.array(outputs)


# Randomize the order of the inputs
# frome: https://stackoverflow.com/a/37710486/2020087
# TODO this might be a better way to do the randomization https://stackoverflow.com/a/30633632
num_inputs = len(inputs)
randomize = np.arange(num_inputs)

np.random.shuffle(randomize)

inputs = inputs[randomize]
outputs = outputs[randomize]

# TODO are we splitting each group of 119 samples or just rows of data?
# --> group of samples (I've called it a recording above)
# spit the data into three bins: training, testing and validation
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)


inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

# build the model and train it
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(15, activation='softmax'))
model.add(tf.keras.layers.Dense(NUM_GESTURES))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(inputs_train, outputs_train, epochs=600, batch_size=1, validation_data=(inputs_validate, outputs_validate))

# graph the loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g.', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# graph the loss again skipping a bit of the start
SKIP = 100
plt.plot(epochs[SKIP:], loss[SKIP:], 'g.', label='Training loss')
plt.plot(epochs[SKIP:], val_loss[SKIP:], 'b.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# graph of mean absolute error
mae = history.history['mae']
val_mae = history.history['val_mae']
plt.plot(epochs[SKIP:], mae[SKIP:], 'g.', label='Training MAE')
plt.plot(epochs[SKIP:], val_mae[SKIP:], 'b.', label='Validation MAE')
plt.title('Training and validation mean absolute error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()


# use the model to predict the test inputs
predictions = model.predict(inputs_test)

# print the predictions and the expected ouputs
print(predictions)
print(outputs_test)


# Plot the predictions along with to the test data
plt.clf()
plt.title('Training data predicted vs actual values')
plt.plot(inputs_test, outputs_test, 'b.', label='Actual')
plt.plot(inputs_test, predictions, 'r.', label='Predicted')
# plt.legend()
plt.show()

# Convert Trained Model to Tensor Flow Light

The next cell converts the model to TFlite format. It also creates a quantized model, that we'll ignore for now. The size in bytes of each model is also printed out.

In [0]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("gesture_model.tflite", "wb").write(tflite_model)
# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
# Save the model to disk
open("gesture_model_quantized.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("gesture_model.tflite")
print("Basic model is %d bytes" % basic_model_size)
quantized_model_size = os.path.getsize("gesture_model_quantized.tflite")
print("Quantized model is %d bytes" % quantized_model_size)
difference = basic_model_size - quantized_model_size
print("Difference is %d bytes" % difference)
  
  

## Encode the Model in an Arduino Header File 

The next cell creates a constant byte array that contains the TFlite model. Import it as a tab with the sketch below.

In [0]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel. Double click model.h to download the file.")

# Classifying IMU Data

1. Download https://github.com/sandeepmistry/aimldevfest-workshop-2019/blob/master/ArduinoSketches/IMU_Classifier/IMU_Classifier.ino sketch
1. Create a new tab named `model.h`, and place the `model.h` generated in the previous code cell inside it.
1. Upload the sketch
1. Open the Serial Monitor: `Tools -> Serial Monitor`
1. Press the button, and perform a gesture
1. The confidence of each gesture will be printed to the Serial Monitor (0 -> low confidence, 1 -> high confidence)

# Add More Gestures

Now that you have this working... Load the code to record gestures. Create more CSV files with gestures. Retrain the model. Load the new model back onto the Arduino.

Note: you'll need to edit the code to add the names of the new geture files.

## Arduino BLE Keyboard

TODO Arduino / BLE HID code goes here
